### Math 395 Learning from Large Data Sets, Hope College, Fall 2024

# Assignment 4 - Logistic regression on the wine quality data to predict wine type

First download the data set from the following url:

https://archive.ics.uci.edu/dataset/186/wine+quality

and then unzip it to make a subfolder, called winequality, under your Python working directory (where you have been storing the python scripts from this course).

In this assignment, you are asked to fit a variety of logistic regression models on the wine quality data set, with wine type (categorical variable) as the target variable.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## First, let us load the two types of wine (red and white) into Python separately. 

We will combine them together later to produce a single data set (with wine type as a categorical variable).

In [ ]:
redwine = pd.read_table("winequality/winequality-red.csv", sep=';')
redwine.head()

In [ ]:
redwine.info()

In [ ]:
redwine.describe()

In [ ]:
whitewine = pd.read_table("winequality/winequality-white.csv", sep=';')
whitewine

Merge the two types of wine to create an expanded data set

In [ ]:
allwine = pd.concat([redwine, whitewine], ignore_index=True)
allwine

Define a new variable to record the wine type (1 : red, 0 : white)

In [ ]:
winetype = np.concatenate((np.ones(redwine.shape[0]), np.zeros(whitewine.shape[0])))
allwine["winetype"] = winetype
allwine

## Do all of the following now. 

We are not splitting the training data into two parts this time, i.e., we use the whole data set for training. Keep in mind that all the errors to be computed below (except the regularized model) are training errors. 

### (a) Visualize the target variable (wine type) and a few (at least 2) other variables of your choice individually

### (b) Make a few scatterplots between wine type and each of several (at least 2) features you think would be best predictors

Fit a logistic regression model with the best predictor you found. Plot the model along with the data. What is the training error? 

### (c) Check multicollinearity among all the features (including wine quality this time), through correlation matrix (which reveals pairwise correlations) and the condition number (overall correlation among all features). 

Comment on your findings. 

Standardize all the features.

### (d) Perform logistic regression on the training data with all features included (we call this the full model). 

Compute the misclassification error for the full model on the training data.

Plot the feature coefficients and label those largest (in magnitude) with the feature names. Which features seem to be the most useful predictors?

Fit a logistic regression model with the two most important features (in terms of magnitude of coefficients). Report the training error and plot the decision boundary.

## (e) Fit a $\ell_1$ regularized logistic regression model with the optimal alpha parameter (which is found by using cross validation) and apply it to the full training data. 

What is the training error of the validated model?

Which predictors are kept and which ones are excluded from the full model? Which features are the most important (in terms of magnitude of coefficients)?

Fit a logistic regression model with the top two features. Report the training error and plot the decision boundary.

For the above model (with top two features), find the confusion matrix and use it to compute the precision, recall and F1 score.

## (f) Summarize your findings.